In [3]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import datetime  # For datetime objects
import os
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import backtrader as bt

# Configuring Global Variables

In [4]:
datapath = './orcl-1995-2014.txt'
data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2000, 12, 31),
        reverse=False)

In [5]:
def print_value(cerebro):
    print("Value Before Running Algorithm ", cerebro.broker.getvalue())
    cerebro.run()
    print("Cache After Running Algorithm ", cerebro.broker.getvalue())

# Runing An Empty Algorithm On No Data

In [6]:
cerebro = bt.Cerebro()
print_value(cerebro)

Value Before Running Algorithm  10000.0
Cache After Running Algorithm  10000.0


# Setting Money

In [7]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.broker.setcash(1e+5)
print_value(cerebro)

Value Before Running Algorithm  100000.0
Cache After Running Algorithm  100000.0


# Defining The Algorithm

In [21]:
class TestStragy(bt.Strategy):
    params = (
    ('myparam', 27),
    ('exitbars', 5),
    )
    def log(self, txt, dt = None):
        '''
        txt : Message
        dt : Date, if None is given will try and use the time stamp on the 
        '''
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))
    
    def __init__(self) :
        
        self.dataclose = self.datas[0].close
#         We added the order attribute in order to check if we have an ongoing order or not
        self.order = None
    
        self.buyprice = None
        self.buycomm  = None 
        
    
    def notify_order(self, order):
        '''
        if there is a change in a order, the order is given to this function so that furthur desicions can be made
        '''
        if  order.status in [order.Submitted, order.Accepted] : 
            return 
        if order.status in [order.Completed] :
            if order.isbuy():
                self.log('BUY Executed, Price : %2f, Cost : %2f, Comm : %2f' % 
                        (order.executed.price, 
                         order.executed.value, 
                         order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm  = order.executed.comm
            elif order.issell():
                self.log('Sell Executed, Price : %2f, Cost : %2f, Comm : %2f' % 
                        (order.executed.price, 
                         order.executed.value, 
                         order.executed.comm))
#             we'll save on which step something was executed
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected] :
            self.log('Order Canceled/Margin/Rejected')
        self.order = None
    
    
    def next(self):
        self.log('Close, %2f' % self.dataclose[0])
        
#         Check if we are already in the market
        if self.order:
            return
        
        # The Heart Of Our Algorithm
        
#         Check if we had bought any thing before or not
        if not self.position : 
            
            if self.dataclose[0] < self.dataclose[-1]  : 

                if self.dataclose[-1] < self.dataclose[-2] :     
                    self.log('BUY Created, %2f' % self.dataclose[0])
                    self.buy()
        
        else :
            
            if len(self) > self.bar_executed + 5 :
                
                self.log('SELL Created, %2f' % self.dataclose[0])
                
                self.order = self.sell()
                

            
            
                
        
                
    

In [22]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(TestStragy, myparam=20, exitbars=7)
# Setting the commision to 0.1%
cerebro.broker.setcommission(1e-3)
cerebro.addsizer

In [23]:
print_value(cerebro)

Value Before Running Algorithm  10000.0
2000-01-03, Close, 26.270000
2000-01-04, Close, 23.950000
2000-01-05, Close, 22.680000
2000-01-05, BUY Created, 22.680000
2000-01-06, BUY Executed, Price : 22.270000, Cost : 22.270000, Comm : 0.022270
2000-01-06, Close, 21.350000
2000-01-07, Close, 22.990000
2000-01-10, Close, 25.740000
2000-01-11, Close, 24.990000
2000-01-12, Close, 23.490000
2000-01-13, Close, 23.360000
2000-01-14, Close, 23.750000
2000-01-14, SELL Created, 23.750000
2000-01-18, Sell Executed, Price : 23.990000, Cost : 22.270000, Comm : 0.023990
2000-01-18, Close, 24.740000
2000-01-19, Close, 25.410000
2000-01-20, Close, 26.350000
2000-01-21, Close, 26.550000
2000-01-24, Close, 24.100000
2000-01-25, Close, 25.100000
2000-01-26, Close, 24.490000
2000-01-27, Close, 23.040000
2000-01-27, BUY Created, 23.040000
2000-01-28, BUY Executed, Price : 22.900000, Cost : 22.900000, Comm : 0.022900
2000-01-28, Close, 21.070000
2000-01-31, Close, 22.220000
2000-02-01, Close, 24.020000
2000-02